<a href="https://colab.research.google.com/github/gulamkibria775/Data_science_project/blob/main/Data_science_project/Image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/driver')

Mounted at /content/driver


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
data_path=os.listdir("/content/driver/MyDrive/All_data_file/rooms")
room_types=os.listdir("/content/driver/MyDrive/All_data_file/rooms")
print(room_types)

['bed_room', 'living_room', 'dining_room']


In [ ]:
rooms=[]
for item in room_types:
  all_rooms=os.listdir('/content/driver/MyDrive/All_data_file/rooms'+'/'+item)
  for room in all_rooms:
    rooms.append((item,str('/content/driver/MyDrive/All_data_file/rooms'+'/'+item)+'/'+room))
    print(rooms[:1])

In [ ]:
rooms


In [4]:
df=pd.DataFrame(data=rooms,columns=["room type","image"])
df.head()

,room type,image
0,bed_room,/content/driver/MyDrive/All_data_file/rooms/be...
1,bed_room,/content/driver/MyDrive/All_data_file/rooms/be...
2,bed_room,/content/driver/MyDrive/All_data_file/rooms/be...
3,bed_room,/content/driver/MyDrive/All_data_file/rooms/be...
4,bed_room,/content/driver/MyDrive/All_data_file/rooms/be...


In [61]:
room_count=df['room type'].value_counts()
room_count

living_room    34
bed_room       12
dining_room    11
Name: room type, dtype: int64

In [5]:
import cv2
path="/content/driver/MyDrive/All_data_file/rooms"
im_size=60
images=[]
labels=[]
for i in room_types:
  data_path=path +'/'+str(i)
  filenames=[i for i in os.listdir(data_path)]
  for f in filenames:
        img = cv2.imread(data_path + '/' + f)  # reading that image as array
        #print(img)  # will get the image as an array
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)


In [6]:
images = np.array(images)

images.shape

(57, 60, 60, 3)

In [7]:
images = images.astype('float32') / 255.0

In [9]:
images.shape

(57, 60, 60, 3)

In [10]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder


y=df['room type'].values
print(y[:5])

['bed_room' 'bed_room' 'bed_room' 'bed_room' 'bed_room']


In [11]:
y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)



[0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1]


In [18]:
y=y.reshape(-1,1)
onehotencoder = OneHotEncoder()  #Converted  scalar output into vector output where the correct class will be 1 and other will be 0
Y= onehotencoder.fit_transform(y)
Y.shape

# from sklearn.compose import ColumnTransformer
# ct = ColumnTransformer([("Name_Of_Your_Step", OneHotEncoder(),[0])], remainder="passthrough") # The last arg ([0]) is the list of columns you want to transform in this step
# y=ct.fit_transform(y)
# y.shape
Y.shape


(57, 3)

In [19]:
Y.shape


(57, 3)

In [20]:
Y.shape

(57, 3)

In [21]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, Y = shuffle(images, Y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.05, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(54, 60, 60, 3)
(54, 3)
(3, 60, 60, 3)
(3, 3)


In [22]:
Y.shape

(57, 3)

In [23]:
import tensorflow as tf
# Number of classes, one class for each of 5 flower.
num_classes = 3

# flattened imge
n_input = 10800


# architecture hyper-parameter
learning_rate = 0.001
training_iters = 10
batch_size = 16
display_step = 20

In [27]:
img_size=60
num_channels=3
tf.compat.v1.disable_eager_execution()
# import tensorflow.compat.v1 as tf
#x = tf.placeholder(tf.float32, [None, n_input])
x = tf.compat.v1.placeholder(tf.float32, shape=[None, img_size,img_size,num_channels])  # None,60,60,3

y_ = tf.compat.v1.placeholder(tf.float32, [None, num_classes]) # None,3



print('Shape of placeholder',x.shape, y_.shape)

Shape of placeholder (None, 60, 60, 3) (None, 3)


In [32]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')   # summation = wx+b, Activation
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')
weights = {
    'w1': tf.Variable(tf.random.normal([5, 5, 3, 32]),name='w1'),
    'w2': tf.Variable(tf.random.normal([5, 5, 32, 64]),name='w2'),
    'w3': tf.Variable(tf.random.normal([5, 5, 64, 128]),name='w3'),
    'wd1': tf.Variable(tf.random.normal([8 * 8 * 128, 2048]),name='wd1'),
    'wout': tf.Variable(tf.random.normal([2048, num_classes]),name='wout')
}

biases = {
    'b1': tf.Variable(tf.random.normal([32]),name='b1'),
    'b2': tf.Variable(tf.random.normal([64]),name='b2'),
    'b3': tf.Variable(tf.random.normal([128]),name='b3'),
    'bd1': tf.Variable(tf.random.normal([2048]),name='bd1'),
    'bout': tf.Variable(tf.random.normal([num_classes]),name='bout')
}

In [33]:
def conv_net(x, weights, biases):

    # reshape input to 60x60x3 size
    x = tf.reshape(x, shape=[-1, 60, 60, 3])

    print("###########################################################################")
    print("size of x is")
    print(x.shape)


    conv1 = conv2d(x, weights['w1'], biases['b1'])
    conv1 = maxpool2d(conv1, k=2)
    print("###########################################################################")
    print("size after 1st conv layer is ")
    print(conv1.shape)


    #input is 30*30*32 image
    # Convolution Layer
    conv2 = conv2d(conv1, weights['w2'], biases['b2'])
    conv2 = maxpool2d(conv2, k=2)
    print("###########################################################################")
    print("size after 2nd conv and pooling layer is")
    print(conv2.shape)


    ### third conv layer
    # input is 15*15*64 image
    # Convolution Layer
    conv3 = conv2d(conv2, weights['w3'], biases['b3'])

    conv3 = maxpool2d(conv3, k=2)
    print("###########################################################################")
    print("size after 3rd conv and pooling layer is")
    print(conv3.shape)
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    print("###########################################################################")
    print("shape after flattening the image")
    print(fc1)  #8192 is the output


    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    print("###########################################################################")
    print("shape after fully connected layer")
    print(fc1)


    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term.
    out = tf.add(tf.matmul(fc1, weights['wout']), biases['bout'])
    print("###########################################################################")
    print("Output layer")
    return out

In [34]:
# Create the model
model = conv_net(x, weights, biases)
print(model)

###########################################################################
size of x is
(None, 60, 60, 3)
###########################################################################
size after 1st conv layer is 
(None, 30, 30, 32)
###########################################################################
size after 2nd conv and pooling layer is
(None, 15, 15, 64)
###########################################################################
size after 3rd conv and pooling layer is
(None, 8, 8, 128)
###########################################################################
shape after flattening the image
Tensor("Reshape_1:0", shape=(None, 8192), dtype=float32)
###########################################################################
shape after fully connected layer
Tensor("Relu_3:0", shape=(None, 2048), dtype=float32)
###########################################################################
Output layer
Tensor("Add_1:0", shape=(None, 3), dtype=float32)


In [38]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y_))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [40]:
init = tf.compat.v1.global_variables_initializer()

In [41]:
# cost_history=[]
# n_epochs =10
# # the execution
# sess = tf.Session()
# sess.run(init)

# train_y=train_y.toarray()

cost_history=[]
n_epochs =5
# the execution
sess = tf.compat.v1.Session()
sess.run(init)

train_y = train_y.toarray()

for i in range(n_epochs):
    a, c = sess.run([optimizer, cost], feed_dict={x: train_x, y_: train_y})  #working
    cost_history = np.append(cost_history,c)  # working
    print('epoch : ', i,  ' - ', 'cost: ', c)

epoch :  0  -  cost:  11610616.0
epoch :  1  -  cost:  2119775.0
epoch :  2  -  cost:  4474184.5
epoch :  3  -  cost:  4183803.8
epoch :  4  -  cost:  5166564.0


In [42]:
test_y=test_y.todense()  #working solution of ValueError: setting an array element with a sequence.
#print(test_y)

correct_prediction = tf.equal(tf.argmax(model,1), tf.argmax(y_,1))
correct_prediction

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy


# retrun the accuracy on the test set.
print("Accuracy: ", sess.run(accuracy, feed_dict={x: test_x, y_:test_y}))

Accuracy:  0.33333334


In [93]:
tf.__version__

'2.15.0'